# model 3

In [1]:
from abc import ABC
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from preprocessing import SentencesEmbeddingDataset
from model2_nn import train_and_plot
from torch.nn.utils import rnn
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.optim import Adam
from train_loop_model3 import train_and_plot_LSTM
from utils import remove_padding
from model3_comp import LSTM_NER_NN
from utils import print_epoch_details, remove_padding
from torch.nn.functional import one_hot

In [3]:
ds = SentencesEmbeddingDataset(vec_dim=500, 
                               list_embedding_paths=['glove-twitter-200', 'word2vec-google-news-300'],
                               list_vec_dims=[200, 300]
                               )


preparing embedding...


In [4]:
batch_size = 32
train_dataloader, dev_dataloader, test_dataloader = ds.get_data_loaders(batch_size=batch_size)

In [5]:
ds.embedding_model=None

In [6]:
torch.save((train_dataloader, dev_dataloader, test_dataloader), 'concated_ds.pkl')

In [12]:
train_dataloader, dev_dataloader, test_dataloader = torch.load('concated_ds.pkl')

In [13]:
train_loader

In [ ]:
ds.vec_dim=300

In [ ]:
batch_size = 32
train_loader, dev_loader = ds.get_data_loaders(batch_size=batch_size)

In [5]:
from model3_comp import main
main()
# run(
#     NER_dataset=ds,
#     embedding_name="concated",
#     vec_dim=500,
#     hidden_dim=256,
#     dropout=0.4,
#     class_weights=torch.tensor([0.1, 0.9]),
#     loss_func=nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9])),
#     loss_func_name='CrossEntropy',
# )

preparing embedding...
----------------------------------------------------------
embedding_name='glove-wiki-gigaword-300' | hidden_dim=256 | dropout=0.2                                 
class_weights=tensor([0.1000, 0.9000]) | loss_func=CrossEntropyLoss()
No GPU available, training on CPU.


# max_pull

In [16]:
# pool of square window of size=3, stride=2
m = nn.MaxPool1d(2, stride=2)
# pool of non-square window
# m = nn.MaxPool2d((3, 2))
input = torch.randn(32, 320)
output = m(input)
input.size(), output.size()

(torch.Size([32, 320]), torch.Size([32, 220]))

# data

In [ ]:
import pandas as pd
df = pd.read_csv('data/ner.csv')
df = df.drop('POS', 1)
df['Sentence'] = df['Sentence'].apply(lambda x: x.split())
df['Tag'] = df['Tag'].apply(lambda x: x.strip("[]").replace("'", '').split(', '))
df['len_sent'] = df['Sentence'].apply(lambda x: len(x))
df['len_tag'] = df['Tag'].apply(lambda x: len(x))
mask = (df['len_sent'] == df['len_tag'])
df = df[mask]
df = df.drop([
    5699, 8331, 9158, 26395, 26399, 28978, 29163, 31422, 31423, 31426, 31427, 
    33653, 34217, 34220, 39286, 39290, 43339, 47182
])

with open('data/my_dataset', 'w') as f:
    for i in df.iterrows():
        for word, tag in zip(i[1]['Sentence'], i[1]['Tag']):
            f.write(f'{str(word)}\t{tag}\n')
        f.write('\n')

In [ ]:
i[0]

47958

In [ ]:
len(df)

47937

# data- CONLL Corpora (2003)

In [8]:
import pandas as pd
df = pd.read_csv('data/train.txt', sep=' ', names=['word', 'irr1', 'irr2', 'tag'])
df = df.drop(['irr1', 'irr2'], 1)

In [11]:
def load_sentences(filepath):

    final = []
    sentences = []

    with open(filepath, 'r') as f:
        
        for line in f.readlines():
            
            if (line == ('-DOCSTART- -X- -X- O\n') or line == '\n'):
                if len(sentences) > 0:
                    final.append(sentences)
                    sentences = []
            else:
                l = line.split(' ')
                sentences.append((l[0], l[3].strip('\n')))
    
    return final

In [13]:
sentences = load_sentences('data/train.txt')
with open('data/my_train.txt', 'w') as f:
    for sent in sentences:
        for word, tag in sent:
            f.write(f'{str(word)}\t{tag}\n')
        f.write('\n')

In [14]:
sentences = load_sentences('data/valid.txt')
with open('data/my_valid.txt', 'w') as f:
    for sent in sentences:
        for word, tag in sent:
            f.write(f'{str(word)}\t{tag}\n')
        f.write('\n')

In [ ]:
def write_to_tagged_file(y_pred):
    empty_lines = ['\t', '']
    with open('data/test.untagged', 'r') as untag:
        with open('data/test.tagged', 'w') as tag:
            for y in y_pred:
                line = ''
                while line in empty_lines:
                    line = untag.readline()
                    if line in empty_lines:
                        tag.write('\n')
                    line = untag.readline()
                tag.write(f"{line}\t{y}\n")

In [9]:
empty_lines = ['\t', '']
with open('data/test.untagged', 'r') as untag:
    with open('data/test.tagged', 'w') as tag:
        line = untag.readline()
        print(type(line))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1247: character maps to <undefined>

In [17]:
import pandas as pd
from sklearn.metrics import f1_score
df = pd.read_csv('comp_206014482_316375872.tagged', sep='\t', names=['word', 'real', 'pred'])
df['real'] = df['real'].apply(lambda x: 0 if x == 'O' else 1)
df['pred'] = df['pred'].apply(lambda x: 0 if x == 'O' else 1)
df

,word,real,pred
0,Stabilized,0,0
1,approach,0,0
2,or,0,0
3,not,0,0
4,?,0,0
...,...,...,...
5178,show,0,0
5179,on,0,0
5180,Discivery,1,0
5181,Channel,1,0


In [18]:
f1_score(df['real'], df['pred'])

0.11144883485309016